With numpy information gain

In [1]:
import math
def entropy(y):
    class_counts = {}
    for label in y:
        class_counts[label] = class_counts.get(label, 0) + 1
    entropy = 0.0
    for label in class_counts:
        p = class_counts[label] / len(y)
        entropy -= p * math.log2(p)
    return entropy

def information_gain(y, mask, feature_value):
    # Split y into two parts
    y_left = [y[i] for i in range(len(y)) if mask[i] == feature_value]
    y_right = [y[i] for i in range(len(y)) if mask[i] != feature_value]
    p = len(y_left) / len(y)
    info_gain = entropy(y) - p * entropy(y_left) - (1 - p) * entropy(y_right)
    return info_gain

In [ ]:
def build_tree(X, y, features):
    # If only one class left, or no feature to split, return a leaf node
    if len(set(y)) == 1 or len(features) == 0:
        return max(set(y), key=y.count)
    
    # Select the best feature to split
    best_feature = None
    best_gain = 0
    for feature in features:
        mask = [x[feature] for x in X]
        unique_values = set(mask)
        for value in unique_values:
            gain = information_gain(y, mask, value)
            if gain > best_gain:
                best_gain = gain
                best_feature = (feature, value)

    if best_feature is None:
        return max(set(y), key=y.count)
    
    # Split dataset
    left_mask = [x[best_feature[0]] == best_feature[1] for x in X]
    right_mask = [x[best_feature[0]] != best_feature[1] for x in X]
    left_X = [X[i] for i in range(len(X)) if left_mask[i]]
    right_X = [X[i] for i in range(len(X)) if right_mask[i]]
    left_y = [y[i] for i in range(len(y)) if left_mask[i]]
    right_y = [y[i] for i in range(len(y)) if right_mask[i]]
    
    # Recursively build left and right branches
    left_branch = build_tree(left_X, left_y, features - {best_feature[0]})
    right_branch = build_tree(right_X, right_y, features - {best_feature[0]})

    return {best_feature: {'left': left_branch, 'right': right_branch}}


In [ ]:
def predict(tree, x):
    for (feature, value), branches in tree.items():
        if x[feature] == value:
            if isinstance(branches['left'], dict):
                return predict(branches['left'], x)
            else:
                return branches['left']
        else:
            if isinstance(branches['right'], dict):
                return predict(branches['right'], x)
            else:
                return branches['right']


In [ ]:
# Sample multi-class dataset
X = [[0, 1], [1, 0], [1, 1], [0, 0], [1, 1], [1, 0]]
y = [0, 1, 2, 0, 2, 1]  # Target labels with more than two classes

# Build the decision tree
features = set(range(len(X[0])))  # Assuming all features are numerical
tree = build_tree(X, y, features)

# Predict for a new instance
sample = [1, 0]
prediction = predict(tree, sample)
print("Prediction:", prediction)


In [2]:
class TreeNode:
    def __init__(self, feature=None, value=None, left=None, right=None, label=None):
        self.feature = feature  # Feature index on which to split
        self.value = value      # Value of the feature to split on
        self.left = left        # Left child (TreeNode)
        self.right = right      # Right child (TreeNode)
        self.label = label      # Label to predict if it's a leaf node

In [3]:
def build_tree(X, y, features):
    # If only one class left, or no feature to split, return a leaf node
    if len(set(y)) == 1 or len(features) == 0:
        return TreeNode(label=max(set(y), key=y.count))
    
    # Select the best feature to split
    best_feature = None
    best_gain = 0
    for feature in features:
        mask = [x[feature] for x in X]
        unique_values = set(mask)
        for value in unique_values:
            gain = information_gain(y, mask, value)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_value = value

    if best_feature is None:
        return TreeNode(label=max(set(y), key=y.count))
    
    # Split dataset
    left_indices = [i for i in range(len(X)) if X[i][best_feature] == best_value]
    right_indices = [i for i in range(len(X)) if X[i][best_feature] != best_value]

    left_X = [X[i] for i in left_indices]
    right_X = [X[i] for i in right_indices]
    left_y = [y[i] for i in left_indices]
    right_y = [y[i] for i in right_indices]
    
    # Recursively build left and right branches
    left_branch = build_tree(left_X, left_y, features - {best_feature})
    right_branch = build_tree(right_X, right_y, features - {best_feature})

    return TreeNode(feature=best_feature, value=best_value, left=left_branch, right=right_branch)


In [4]:
def predict(tree_node, x):
    if tree_node.label is not None:
        return tree_node.label
    if x[tree_node.feature] == tree_node.value:
        return predict(tree_node.left, x)
    else:
        return predict(tree_node.right, x)

In [ ]:
def gini_impurity(y):
    class_counts = {}
    for label in y:
        class_counts[label] = class_counts.get(label, 0) + 1
    impurity = 1
    for label in class_counts:
        prob_of_lbl = class_counts[label] / float(len(y))
        impurity -= prob_of_lbl**2
    return impurity

def gini_gain(y, mask, feature_value):
    # Split y into two parts
    y_left = [y[i] for i in range(len(y)) if mask[i] == feature_value]
    y_right = [y[i] for i in range(len(y)) if mask[i] != feature_value]
    p = len(y_left) / len(y)
    gain = gini_impurity(y) - p * gini_impurity(y_left) - (1 - p) * gini_impurity(y_right)
    return gain


In [6]:
# Sample multi-class dataset
X = [[0, 1], [1, 0], [1, 1], [0, 0], [1, 1], [1, 0]]
y = [0, 1, 2, 0, 2, 1]  # Target labels with more than two classes

# Build the decision tree
features = set(range(len(X[0])))  # Assuming all features are numerical
tree = build_tree(X, y, features)

# Predict for a new instance
sample = [1, 1]
prediction = predict(tree, sample)
print("Prediction:", prediction)

Prediction: 2
